In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
%matplotlib inline

In [2]:
def compute_similarity_transform(A, B):
    """
    Compute the similarity transformation (s, R, t) that best aligns each point in A
    to the corresponding point in B, including uniform scaling.
    """
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # Ensure a proper rotation by enforcing a right-handed coordinate system
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = np.dot(Vt.T, U.T)

    # Compute scaling
    scale = np.sum(S) / np.sum(AA ** 2) # Umeyama
    scale = (np.sum(BB ** 2) / np.sum(AA ** 2))**0.5 # wiki

    # Compute translation
    t = centroid_B - scale * np.dot(R, centroid_A)

    return scale, R, t

In [3]:
def apply_similarity_transformation(points, s, R, t):
    """
    Apply the computed similarity transformation (s, R, t) to the points.
    """
    return s * np.dot(points, R.T) + t

In [4]:
def get_numpy_points_from_input(df):
    # Extract x, y, z values as NumPy arrays
    x_values = df['x'].values
    y_values = df['y'].values
    z_values = df['z'].values

    # Create a NumPy array from these values
    np_array = np.array([x_values, y_values, z_values])
    
    return np_array

In [5]:
def plot_as_scatter(np_array):
    # Plotting the 3D scatter plot
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot
    ax.scatter(np_array[0], np_array[1], np_array[2], c='b', marker='o')

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

    plt.title('3D Scatter plot of x, y, and z values')
    plt.show()

In [6]:
before_path = 'data/dataroot/products/eda-fduh0l8m/section'
after_path = 'data/dataroot/products/eda-xwtafb42/section'

In [12]:
def before_points_alignment(before_path, after_path):
    for before_folder_name in os.listdir(before_path):
        before_folder_path = os.path.join(before_path, before_folder_name)
        
        after_folder_path = os.path.join(after_path, before_folder_name)
        
        if os.path.isdir(before_folder_path) and before_folder_name.startswith('chr'):
            before_file_path = os.path.join(before_folder_path, 'structure', '100kb', 'structure-with-tracks.csv')
            
            after_file_path = os.path.join(after_folder_path, 'structure', '100kb', 'structure-with-tracks.csv')
            
            if os.path.exists(before_file_path) and os.path.exists(after_file_path):
                before_df = pd.read_csv(before_file_path)                
                after_df = pd.read_csv(after_file_path)
                
                # getting the coords in numpy array
                beforePoints = get_numpy_points_from_input(before_df)
                afterPoints = get_numpy_points_from_input(after_df)
                
                # Compute the similarity transformation that aligns points A to points B
                s, R, t = compute_similarity_transform(beforePoints, afterPoints)
                
                print(s)
                print(len(R))
                print(len(t))
                print(len(beforePoints[0]), len(afterPoints))
                
#                 # Apply the similarity transformation to points A
#                 transformed_beforePoints = apply_similarity_transformation(beforePoints, s, R, t)
                
#                 # change the xyz columns to to old
#                 before_df = before_df.rename(columns={'x': 'x_old', 'y': 'y_old', 'z': 'z_old'})
                
# #                 print(len(transformed_beforePoints))
# #                 print(transformed_beforePoints[0])
                
#                 # add the new points
#                 before_df['x'] = transformed_beforePoints[0].tolist()
#                 before_df['y'] = transformed_beforePoints[1].tolist()
#                 before_df['z'] = transformed_beforePoints[2].tolist()
                
# #                 print(before_df.head())
                
#                 save_file_path = os.path.join(before_folder_path, 'structure', '100kb', 'structure-with-rotation.csv')
                
#                 before_df.to_csv(save_file_path, index=False)
                
#                 print(f"{save_file_path} is saved") 
            break
                
                
                

        
before_points_alignment(before_path, after_path)

1.4522973768343221
983
983
983 3
